## <B>Calibrate EAR data</B>

In [ ]:
import pandas as pd

# Variables to use for filenames
name = "37"

# List of movie names
movies = ["기생충1", "기생충2", "낭닥1", "낭닥2", "코1", "코2", "마녀1", "마녀2", "레버1", "레버2"]

# Processing data by file
for movie in movies:
    # Create a file path
    file = f'{name} {movie} 보정.csv'
    
    # Read a file
    data = pd.read_csv(file)
    
    # Convert timestamp to seconds
    def convert_to_seconds(timestamp):
        minutes, seconds = map(float, timestamp.split(":"))
        return minutes * 60 + seconds

    data['Seconds'] = data['Timestamp'].apply(convert_to_seconds)
    
    # Group by 0.1 second interval
    data['Time_Group'] = (data['Seconds'] * 10).astype(int) / 10
    
    # Calculate EAR averages at 0.1 second intervals
    result = data.groupby('Time_Group')['EAR'].mean().reset_index()
    
    # Save the results
    output_file = f'{name} {movie} 평균_EAR.csv'
    result.to_csv(output_file, index=False)
    print(f"Processed: {output_file}")

## <B>EAR data calibration (for special cases of time data)</B>

In [ ]:
import pandas as pd

# File name
file = "37 낭닥1 보정.csv"

# Convert to seconds function (automatic calculation based on first time)
def convert_to_seconds_with_auto_start(timestamp, initial_time):
    initial_minutes, initial_seconds = map(float, initial_time.split(":"))
    initial_total_seconds = initial_minutes * 60 + initial_seconds

    current_minutes, current_seconds = map(float, timestamp.split(":"))
    current_total_seconds = current_minutes * 60 + current_seconds

    # Handling boundary crossings (current_total_seconds < initial_total_seconds)
    if current_total_seconds < initial_total_seconds % 3600:
        current_total_seconds += 3600  # 1시간 추가

    return current_total_seconds

# Read a file
data = pd.read_csv(file)

# Handling exceptions when data is empty
if data.empty:
    print(f"Empty file: {file}")
else:
    # Set the initial time based on the first time value
    initial_time = data['Timestamp'].iloc[0]

    # Add a ‘Seconds’ column
    data['Seconds'] = data['Timestamp'].apply(lambda t: convert_to_seconds_with_auto_start(t, initial_time))

    # Group by 0.1 second interval
    data['Time_Group'] = (data['Seconds'] * 10).astype(int) / 10

    # Calculate EAR averages at 0.1 second intervals
    result = data.groupby('Time_Group')['EAR'].mean().reset_index()

    # Save the results
    output_file = "37 낭닥1 평균_EAR.csv"
    result.to_csv(output_file, index=False)
    print(f"Processed: {output_file}")

## <B>Eye Openness Ratio During Gory Video Viewing Analysis</B>

In [ ]:
import pandas as pd

name = "37"

movies = ["기생충", "낭닥", "코", "마녀", "레버"]

for movie in movies:
    data1 = pd.read_csv(f'{name} {movie}1 평균_EAR.csv')
    data2 = pd.read_csv(f'{name} {movie}2 평균_EAR.csv')

    count_data3 = data1['EAR'].mean()
    count_data4 = data2['EAR'].mean()

    max_data1 = data1['EAR'].max()
    min_data1 = data1['EAR'].min()
    
    max_data2 = data2['EAR'].max()
    min_data2 = data2['EAR'].min()

    # EAR normalization
    normalized_data1 = (data1['EAR'] / max_data1) * 100
    normalized_data2 = (data2['EAR'] / max_data2) * 100

    normalized_mean_data1 = normalized_data1.mean()
    normalized_mean_data2 = normalized_data2.mean()

    print(f"EAR average in data1.csv: {count_data3}")
    print(f"EAR average in data2.csv: {count_data4}")
    print(f"EAR maximum for data1.csv: {max_data1}")
    print(f"EAR minimum for data1.csv: {min_data1}")
    print(f"EAR maximums in data2.csv: {max_data2}")
    print(f"EAR minimums in data2.csv: {min_data2}")
    print(f"EAR average of normalized data1.csv: {normalized_mean_data1}")
    print(f"EAR average for normalized data2.csv: {normalized_mean_data2}")
    print("---------------------------------------------------------")

## <B>Blink Frequency Analysis</B>

In [ ]:
import pandas as pd

name = "37"

movies = ["기생충", "낭닥", "코", "마녀", "레버"]

for movie in movies:
    data1 = pd.read_csv(f'{name} {movie}1 평균_EAR.csv')
    data2 = pd.read_csv(f'{name} {movie}2 평균_EAR.csv')

    # EAR normalization
    max_data1 = data1['EAR'].max()
    min_data1 = data1['EAR'].min()
    
    max_data2 = data2['EAR'].max()
    min_data2 = data2['EAR'].min()

    data1['Normalized EAR'] = ((data1['EAR'] - min_data1) / (max_data1 - min_data1)) * 100
    data2['Normalized EAR'] = ((data2['EAR'] - min_data2) / (max_data2 - min_data2)) * 100

    data1['EAR 0/1'] = (data1['EAR'] >= 0.25).astype(int)
    data2['EAR 0/1'] = (data2['EAR'] >= 0.25).astype(int)

    # Count the instant of change from 0 to 1
    data1['Transition Count'] = ((data1['EAR 0/1'].shift(1) == 0) & (data1['EAR 0/1'] == 1)).astype(int)
    data2['Transition Count'] = ((data2['EAR 0/1'].shift(1) == 0) & (data2['EAR 0/1'] == 1)).astype(int)

    count_transitions_data1 = data1['Transition Count'].sum()
    count_transitions_data2 = data2['Transition Count'].sum()

    print(f"Count the instantaneous 0 to 1 transitions in data1.csv: {count_transitions_data1}")
    print(f"Count the instantaneous 0 to 1 transitions in data2.csv: {count_transitions_data2}")
    print("---------------------------------------------------------")